In [1]:
from Bio import Entrez

In [2]:
import pprint

In [3]:
import pandas as pd
from pandas.io.json import json_normalize

In [4]:
def search(query):
    Entrez.email = 'sxxize@gmail.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='20',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

In [5]:
def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'sxxize@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [6]:
results = search('nanoparticle')
id_list = results['IdList']
papers = fetch_details(id_list)
for i, paper in enumerate(papers['PubmedArticle']):
    print("%d) %s" % (i+1, paper['MedlineCitation']['Article']['ArticleTitle']))

1) Preferential binding of positive nanoparticles on cell membranes is due to electrostatic interactions: A too simplistic explanation that does not take into account the nanoparticle protein corona.
2) Magnetic nanoparticles to recover cellular organelles and study the time resolved nanoparticle-cell interactome throughout uptake.
3) Peritumoral implantation of hydrogel-containing nanoparticles and losartan for enhanced nanoparticle penetration and antitumor effect.
4) Monitoring of the Cytoskeleton-Dependent Intracellular Trafficking of Fluorescent Iron Oxide Nanoparticles by Nanoparticle Pulse-Chase Experiments in C6 Glioma Cells.
5) Nanoparticle Tracking Analysis for Determination of Hydrodynamic Diameter, Concentration, and Zeta-Potential of Polyplex Nanoparticles.
6) Electrochemiluminescence observing the surface features of Ru-doped silica nanoparticles based on nanoparticle-ultramicroelectrode collision.
7) Programmed Nanoparticle-Loaded Nanoparticles for Deep-Penetrating 3D Ca

In [7]:
papers.keys()

dict_keys(['PubmedBookArticle', 'PubmedArticle'])

In [8]:
papers['PubmedArticle'][0].keys()

dict_keys(['MedlineCitation', 'PubmedData'])

In [9]:
papers['PubmedArticle'][0]['MedlineCitation'].keys()

dict_keys(['KeywordList', 'SpaceFlightMission', 'CitationSubset', 'OtherID', 'OtherAbstract', 'GeneralNote', 'PMID', 'DateCompleted', 'DateRevised', 'Article', 'MedlineJournalInfo', 'ChemicalList', 'MeshHeadingList'])

In [14]:
pprint.pprint(papers['PubmedArticle'][0]['MedlineCitation'])

{'Article': {'Abstract': {'AbstractText': ['The internalization of nanoparticles by cells (and more broadly the nanoparticle/cell interaction) is a crucial issue both for biomedical applications (for the design of nanocarriers with enhanced cellular uptake to reach their intracellular therapeutic targets) and in a nanosafety context (as the internalized dose is one of the key factors in cytotoxicity). Many parameters can influence the nanoparticle/cell interaction, among them, the nanoparticle physico-chemical features, and especially the surface charge. It is generally admitted that positive nanoparticles are more uptaken by cells than neutral or negative nanoparticles. It is supposedly due to favorable electrostatic interactions with negatively charged cell membrane. However, this theory seems too simplistic as it does not consider a fundamental element: the nanoparticle protein corona. Indeed, once introduced in a biological medium nanoparticles adsorb proteins at their surface, for